# **4. **

In [9]:
!pip install dash
!pip install dash-leaflet
!pip install geocoder
!pip install plotly
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from scipy import stats
import dash
from dash import html  # Updated import
from dash import dcc  # Updated import
import dash_leaflet as dl
from dash.dependencies import Input, Output, State
import geocoder
import plotly.express as px
import plotly.graph_objects as go
from geopy.extra.rate_limiter import RateLimiter # to avoid overloading the server
from functools import lru_cache #used to cache the results

In [10]:
# Load the dataframe
df = pd.read_csv('Final_features.csv')
df.head()

,listing_id,listing_heading,listing_type,listing_url,listing_first_date,days_on_market,dealer_id,dealer_name,dealer_street,dealer_city,...,price_history_delimited,distance_to_dealer,location_score,listing_dropoff_date,latitude,longitude,total_vehicles_sold,most_sold_brand,total_revenue,avg_days_on_market
0,f5b663d2-4896-11ef-a15b-8b4423f1ff08,1987 IROC Z 350 Auto T-Tops,Active,https://www.kijiji.ca/v-cars-trucks/edmonton/1...,7/22/2024 0:00,9,11152931,David T's Camaro & Firebird Auto Center,10611 201 St NW,Edmonton,...,"2024-07-22,11500,230860",9.84,5,NaN,53.549233,-113.666536,0.0,NaN,NaN,11
1,a9f565e6-4712-11ef-a23f-79dc25c5e601,NaN,Sold,https://www.redlinemotors.ca/vehicle-details/1...,7/20/2024 0:00,4,11131329,Redlinemotors.Ca,14421 Mark Messier Trail,Edmonton,...,"2024-07-20,8700,48905",8.30,5,7/24/2024 0:00,53.604536,-113.578942,43.0,Ford,428200.0,108
2,03d18c58-253a-11ef-8f33-8905b4a2f000,1989 Cadillac DeVille Sedan FWD,Active,https://www.sweetheartmotor.com/cars/used/1989...,6/7/2024 0:00,54,11161704,Gateway Auto & Rv Sales & Financing Ltd,5404 Gateway Blvd NW,Edmonton,...,"2024-06-07,6000,174640",5.71,4,NaN,53.492482,-113.493143,70.0,Chevrolet,3266527.0,33
3,2d12630b-460e-11ee-89c4-e5b3161d4741,1989 Cadillac DeVille Sedan FWD,Active,https://www.sweetheartmotor.com/cars/used/1989...,8/28/2023 0:00,338,11128265,Credit Angels,5404 Gateway Blvd NW,Edmonton,...,"2023-08-28,4500,174640:2023-09-13,5900,174640:...",5.71,5,NaN,53.492482,-113.493143,34.0,Dodge,243171.0,94
4,53a8e550-47cd-11ef-8587-f926ecb61035,1989 Camaro IROC 350 Auto T-Top,Active,https://www.kijiji.ca/v-cars-trucks/edmonton/1...,7/21/2024 0:00,10,11152931,David T's Camaro & Firebird Auto Center,10611 201 St NW,Edmonton,...,"2024-07-21,8500,288000",9.84,5,NaN,53.549233,-113.666536,0.0,NaN,NaN,11


In [11]:
app = dash.Dash(__name__)
app.layout = html.Div([
    html.Div([
        dcc.Input(id="search-input", type="text", placeholder="Search for a dealership..."),
        html.Button("Search", id="search-button")
    ], style={'display': 'flex', 'justifyContent': 'center', 'marginBottom': '10px'}),

    dcc.Graph(id='plotly-map', figure={},
              config={'scrollZoom': True})  # Add config for zoom on click
])
@app.callback(
    Output('plotly-map', 'figure'),
    [Input("search-button", "n_clicks")],  # Triggered by the search button
    [State("search-input", "value")]     # Get the search term
)
def update_plotly_map(n_clicks, search_term):
    filtered_df = df  # Start with the full dataset

    if search_term:  # If a search term is provided
        # Filter the DataFrame based on the search term (dealer_name)
        filtered_df = df[df['dealer_name'].str.contains(search_term, case=False)]

    fig = px.scatter_mapbox(
        filtered_df,
        lat="latitude",
        lon="longitude",
        hover_name="dealer_name",
        size_max=15,
        zoom=11,
        mapbox_style="open-street-map",
        color="total_vehicles_sold",
        color_continuous_scale="Viridis",
        range_color=[0, 3000],
        custom_data=["dealer_postal_code", "total_vehicles_sold", "most_sold_brand", "total_revenue", "avg_days_on_market"]
    )

    min_marker_size = 12
    max_marker_size = 30
    scaling_factor = (max_marker_size - min_marker_size) / max(filtered_df['total_vehicles_sold'])

    fig.update_traces(
        marker=dict(
            size=min_marker_size + filtered_df['total_vehicles_sold'] * scaling_factor,
            opacity=0.7
        ),
        hovertemplate="<br>".join([
            "<b>%{hovertext}</b>",
            '\n',
            "Dealer Postal Code: %{customdata[0]}",
            "Total Vehicles Sold: %{customdata[1]}",
            "Most Sold Brand: %{customdata[2]}",
            "Total Revenue: ${%{customdata[3]:,.0f}}",
            "Average Days on Market: %{customdata[4]} days"
        ])
    )

    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0},
                      showlegend=True,
                      mapbox=dict(
                          center=dict(lat=53.5461, lon=-113.4938),
                          zoom=9,
                          uirevision='foo'
                      )
                      )

    return fig


if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>